In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
import os

from selenium.webdriver.chrome.options import Options
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
from datetime import datetime
import time
import pandas as pd
import numpy as np
import cx_Oracle
import re
from tqdm import tqdm
from IPython import display

import urllib

In [2]:
naver_df = pd.read_csv('음식점완성2.csv', encoding='cp949')
naver_df = naver_df[['콘텐츠명']]
naver_df['naver_keyword'] = '제주도%20' + naver_df['콘텐츠명']
naver_df['naver_keyword'] = naver_df['naver_keyword'].str.replace(' ', '%20')
naver_df

,콘텐츠명,naver_keyword
0,달빛머금은카페,제주도%20달빛머금은카페
1,해적선,제주도%20해적선
2,영춘식당,제주도%20영춘식당
3,멘도롱 카페,제주도%20멘도롱%20카페
4,열공부엌,제주도%20열공부엌
...,...,...
1575,바다풍경횟집,제주도%20바다풍경횟집
1576,우리회수산,제주도%20우리회수산
1577,제주선,제주도%20제주선
1578,우무깨식당,제주도%20우무깨식당


In [3]:
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
naver_driver = webdriver.Chrome(service=service, options=options)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 75.0MB/s]


In [ ]:
for i, keyword in enumerate(naver_df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {naver_df.shape[0] -1} 행", keyword)
    
    name_list = []
    cate_list = []
    star_list = []
    addr_list = []
    oper_list = []
    tel_list = []
    
    img_list = []
    nic_list = []
    re_list = []
    date_list = []

    try:
        naver_driver.get(f'https://map.naver.com/v5/search/{keyword}')
        naver_driver.implicitly_wait(5)
        
        try:
            naver_driver.switch_to.frame('entryIframe')
            naver_driver.implicitly_wait(5)

            # 가게이름
            if naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text == naver_df.iloc[i]['콘텐츠명']:

                naver_driver.implicitly_wait(5)
                name = naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text
                print(name)
                name_list.append(name)

                # 카테고리 수집
                try:
                    naver_driver.implicitly_wait(5)
                    naver_driver.find_element(By.CLASS_NAME,'DJJvD')
                    cate = naver_driver.find_element(By.CLASS_NAME,'DJJvD').text
                    naver_driver.implicitly_wait(5)
                    print(cate)
                    cate_list.append(cate)

                except:
                    print('카테고리없음')
                    star_list.append('')

                # 별점 수집
                try:
                    naver_driver.implicitly_wait(5)
                    naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF')
                    star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
                    naver_driver.implicitly_wait(5)
                    print(star)
                    star_list.append(star)

                except:
                    print('별점없음')
                    star_list.append('')

                # 주소 수집
                try:
                    naver_driver.implicitly_wait(5)
                    naver_driver.find_element(By.CLASS_NAME,'LDgIH')
                    addr = naver_driver.find_element(By.CLASS_NAME,'LDgIH').text
                    naver_driver.implicitly_wait(5)
                    print(addr)
                    addr_list.append(addr)

                except:
                    print('주소없음')
                    addr_list.append('')

                # 운영시간 수집
                try:
                    naver_driver.implicitly_wait(3)
                    naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/a').send_keys('\n')
                    naver_driver.implicitly_wait(3)
                    oper = naver_driver.find_element(By.CLASS_NAME,'i8cJw').text
                    print(oper)
                    oper_time = naver_driver.find_element(By.CLASS_NAME,'H3ua4').text
                    print(oper_time)
                    oper_list.append(oper+''+oper_time)

                except:
                    print('운영시간없음')
                    oper_list.append('')

                # 전화번호 수집
                try:
                    naver_driver.implicitly_wait(5)
                    naver_driver.find_element(By.CLASS_NAME,'xlx7Q')
                    tel = naver_driver.find_element(By.CLASS_NAME,'xlx7Q').text
                    naver_driver.implicitly_wait(5)
                    print(tel)
                    tel_list.append(tel)

                except:
                    print('전화번호없음')
                    tel_list.append('')

                # 리뷰 수집
                try:
                    naver_driver.implicitly_wait(5)
                    naver_driver.find_element(By.XPATH, "//a[contains(text(), '리뷰')]").click()
                    print('리뷰 클릭')
                        
                    # 더보기 클릭
                    try:
                        while True:
                            naver_driver.implicitly_wait(5)
                            naver_driver.find_element(By.CLASS_NAME,'fvwqf').click()
                            print('더보기 클릭')

                            try:
                                scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                while True:
                                    naver_driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                                    time.sleep(2)

                                    scroll_height = naver_driver.execute_script("return document.body.scrollHeight")

                                    if scroll_location == scroll_height:
                                        break

                                    else:
                                        scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                    naver_driver.implicitly_wait(5)
                                    print('스크롤 내림')

                            except:
                                print('스크롤 안됨')

                            if not naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]'):
                                break
                                
                    except:
                        print('더보기 안됨')
                        
                    # 리뷰 더보기 클릭
                    try:
                        for button in naver_driver.find_elements(By.CSS_SELECTOR,'div.ZZ4OK > a.xHaT3'):
                            button.send_keys('\n')
                            naver_driver.implicitly_wait(3)
                            print('리뷰 더보기 클릭')

                    except:
                        print('리뷰 더보기 없음')

                    # 이미지, 리뷰 소스 가져오기
                    try:
                        naver_driver.find_element(By.CLASS_NAME,'sBWyy')
                        naver_driver.implicitly_wait(5)
                        print('sBWyy 있음')

                        naver_html = naver_driver.page_source
                        naver_soup = BeautifulSoup(naver_html, 'lxml')
                        naver_driver.implicitly_wait(5)
                        print('페이지 소스 가져옴')
                        
                        # 이미지 수집
                        try:
                            img = naver_soup.find('div', attrs = {'id':'ibu_1'})
                            naver_driver.implicitly_wait(5)
                            img_link = re.findall('["](https.*[.].*g)["]', str(img))
                            print(img_link)
                            img_list.append(img_link)

                        except:
                            print('이미지없음')
                            img_list.append('')
                            
                        try:
                            for lst in naver_soup.find_all('li', attrs = {'class':'YeINN'}):
                                try:
                                    lst.find('div', attrs = {'class':'sBWyy'})
                                    print(lst.find('div', attrs = {'class':'sBWyy'}).text)
                                    nic_list.append(lst.find('div', attrs = {'class':'sBWyy'}).text)
                                except:
                                    print('닉네임 없음')
                                    nic_list.append('')

                                try:
                                    naver_driver.implicitly_wait(3)
                                    lst.find('span', attrs = {'class':'zPfVt'})
                                    print(lst.find('span', attrs = {'class':'zPfVt'}).text)
                                    re_list.append(lst.find('span', attrs = {'class':'zPfVt'}).text)
                                except:
                                    print('글 리뷰없음')
                                    re_list.append('')

                                try:
                                    naver_driver.implicitly_wait(3)   
                                    lst.find('time', attrs = {'aria-hidden':'true'})
                                    print(lst.find('time', attrs = {'aria-hidden':'true'}).text)
                                    date_list.append(lst.find('time', attrs = {'aria-hidden':'true'}).text)
                                except:
                                    print('리뷰 날짜 없음')
                                    date_list.append('')
                        except:
                            pass
                        
                    except:
                        print('리뷰없음')
                        img_list.append('')
                        nic_list.append('')
                        re_list.append('')
                        date_list.append('')

                except:
                    print('리뷰 클릭 안됨')
                    img_list.append('')
                    nic_list.append('')
                    re_list.append('')
                    date_list.append('')

            else:
                print('가게없음')
                name_list.append(naver_df.iloc[i]['콘텐츠명'])
                cate_list.append('')
                star_list.append('')
                addr_list.append('')
                oper_list.append('')
                tel_list.append('')
                
        except:
            try:
                naver_driver.switch_to.default_content()
                naver_driver.switch_to.frame('searchIframe')
                naver_driver.implicitly_wait(5)

                # 가게이름 여러개 중 일치하는 것만 클릭
                try:
                    naver_driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/div[2]/a[1]').click()
                    
                    naver_driver.switch_to.default_content()
                    naver_driver.switch_to.frame('entryIframe')
                    naver_driver.implicitly_wait(5)
                    
                    # 가게이름
                    if naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text == naver_df.iloc[i]['콘텐츠명']:

                        naver_driver.implicitly_wait(5)
                        name = naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text
                        print(name)
                        name_list.append(name)

                        # 카테고리 수집
                        try:
                            naver_driver.implicitly_wait(5)
                            naver_driver.find_element(By.CLASS_NAME,'DJJvD')
                            cate = naver_driver.find_element(By.CLASS_NAME,'DJJvD').text
                            naver_driver.implicitly_wait(5)
                            print(cate)
                            cate_list.append(cate)

                        except:
                            print('카테고리없음')
                            star_list.append('')

                        # 별점 수집
                        try:
                            naver_driver.implicitly_wait(5)
                            naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF')
                            star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
                            naver_driver.implicitly_wait(5)
                            print(star)
                            star_list.append(star)

                        except:
                            print('별점없음')
                            star_list.append('')

                        # 주소 수집
                        try:
                            naver_driver.implicitly_wait(5)
                            naver_driver.find_element(By.CLASS_NAME,'LDgIH')
                            addr = naver_driver.find_element(By.CLASS_NAME,'LDgIH').text
                            naver_driver.implicitly_wait(5)
                            print(addr)
                            addr_list.append(addr)

                        except:
                            print('주소없음')
                            addr_list.append('')

                        # 운영시간 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/a').send_keys('\n')
                            naver_driver.implicitly_wait(3)
                            oper = naver_driver.find_element(By.CLASS_NAME,'i8cJw').text
                            print(oper)
                            oper_time = naver_driver.find_element(By.CLASS_NAME,'H3ua4').text
                            print(oper_time)
                            oper_list.append(oper+''+oper_time)

                        except:
                            print('운영시간없음')
                            oper_list.append('')

                        # 전화번호 수집
                        try:
                            naver_driver.implicitly_wait(5)
                            naver_driver.find_element(By.CLASS_NAME,'xlx7Q')
                            tel = naver_driver.find_element(By.CLASS_NAME,'xlx7Q').text
                            naver_driver.implicitly_wait(5)
                            print(tel)
                            tel_list.append(tel)

                        except:
                            print('전화번호없음')
                            tel_list.append('')

                        # 리뷰 수집
                        try:
                            naver_driver.implicitly_wait(5)
                            naver_driver.find_element(By.XPATH, "//a[contains(text(), '리뷰')]").click()
                            print('리뷰 클릭')

                            # 더보기 클릭
                            try:
                                while True:
                                    naver_driver.implicitly_wait(5)
                                    naver_driver.find_element(By.CLASS_NAME,'fvwqf').click()
                                    print('더보기 클릭')

                                    try:
                                        scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                        while True:
                                            naver_driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                                            time.sleep(2)

                                            scroll_height = naver_driver.execute_script("return document.body.scrollHeight")

                                            if scroll_location == scroll_height:
                                                break

                                            else:
                                                scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                            naver_driver.implicitly_wait(5)
                                            print('스크롤 내림')

                                    except:
                                        print('스크롤 안됨')

                                    if not naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]'):
                                        break

                            except:
                                print('더보기 안됨')

                            # 리뷰 더보기 클릭
                            try:
                                for button in naver_driver.find_elements(By.CSS_SELECTOR,'div.ZZ4OK > a.xHaT3'):
                                    button.send_keys('\n')
                                    naver_driver.implicitly_wait(3)
                                    print('리뷰 더보기 클릭')

                            except:
                                print('리뷰 더보기 없음')

                            # 이미지, 리뷰 소스 가져오기
                            try:
                                naver_driver.find_element(By.CLASS_NAME,'sBWyy')
                                naver_driver.implicitly_wait(5)
                                print('sBWyy 있음')

                                naver_html = naver_driver.page_source
                                naver_soup = BeautifulSoup(naver_html, 'lxml')
                                naver_driver.implicitly_wait(5)
                                print('페이지 소스 가져옴')

                                # 이미지 수집
                                try:
                                    img = naver_soup.find('div', attrs = {'id':'ibu_1'})
                                    naver_driver.implicitly_wait(5)
                                    img_link = re.findall('["](https.*[.].*g)["]', str(img))
                                    print(img_link)
                                    img_list.append(img_link)

                                except:
                                    print('이미지없음')
                                    img_list.append('')

                                try:
                                    for lst in naver_soup.find_all('li', attrs = {'class':'YeINN'}):
                                        try:
                                            lst.find('div', attrs = {'class':'sBWyy'})
                                            print(lst.find('div', attrs = {'class':'sBWyy'}).text)
                                            nic_list.append(lst.find('div', attrs = {'class':'sBWyy'}).text)
                                        except:
                                            print('닉네임 없음')
                                            nic_list.append('')

                                        try:
                                            naver_driver.implicitly_wait(3)
                                            lst.find('span', attrs = {'class':'zPfVt'})
                                            print(lst.find('span', attrs = {'class':'zPfVt'}).text)
                                            re_list.append(lst.find('span', attrs = {'class':'zPfVt'}).text)
                                        except:
                                            print('글 리뷰없음')
                                            re_list.append('')

                                        try:
                                            naver_driver.implicitly_wait(3)   
                                            lst.find('time', attrs = {'aria-hidden':'true'})
                                            print(lst.find('time', attrs = {'aria-hidden':'true'}).text)
                                            date_list.append(lst.find('time', attrs = {'aria-hidden':'true'}).text)
                                        except:
                                            print('리뷰 날짜 없음')
                                            date_list.append('')
                                except:
                                    pass

                            except:
                                print('리뷰없음')
                                img_list.append('')
                                nic_list.append('')
                                re_list.append('')
                                date_list.append('')

                        except:
                            print('리뷰 클릭 안됨')
                            img_list.append('')
                            nic_list.append('')
                            re_list.append('')
                            date_list.append('')
                            
                    else:
                        print('가게없음')
                        name_list.append(naver_df.iloc[i]['콘텐츠명'])
                        cate_list.append('')
                        star_list.append('')
                        addr_list.append('')
                        oper_list.append('')
                        tel_list.append('')

                except:
                    print('가게없음')
                    name_list.append(naver_df.iloc[i]['콘텐츠명'])
                    cate_list.append('')
                    star_list.append('')
                    addr_list.append('')
                    oper_list.append('')
                    tel_list.append('')
                    
            except:
                print('가게 정보 없음')
                name_list.append(naver_df.iloc[i]['콘텐츠명'])
                cate_list.append('')
                star_list.append('')
                addr_list.append('')
                oper_list.append('')
                tel_list.append('')

    except:
        print('가게 정보 없음')
        name_list.append(naver_df.iloc[i]['콘텐츠명'])
        cate_list.append('')
        star_list.append('')
        addr_list.append('')
        oper_list.append('')
        tel_list.append('')

이번에 찾을 키워드 : 0 / 1579 행 제주도%20달빛머금은카페
가게없음
이번에 찾을 키워드 : 1 / 1579 행 제주도%20해적선
해적선
생선회
별점없음
제주 제주시 추자면 추자로 42-1 1층
운영시간없음
0507-1350-7112
리뷰 클릭
더보기 안됨
리뷰없음
이번에 찾을 키워드 : 2 / 1579 행 제주도%20영춘식당
가게없음
이번에 찾을 키워드 : 3 / 1579 행 제주도%20멘도롱%20카페
가게없음
이번에 찾을 키워드 : 4 / 1579 행 제주도%20열공부엌
가게없음
이번에 찾을 키워드 : 5 / 1579 행 제주도%20황금섬숯불갈비
황금섬숯불갈비
육류,고기요리
별점
4.25/5
제주 제주시 추자면 추자로 36
운영시간없음
064-742-0478
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
['https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190731_123%2F15645482684150EmEX_JPEG%2FDm4O6Zfevi1WF37LsnqUNF7M.JPG.jpg']
pinky386
추자도에서 가장 큰 음식점인듯
밑반찬도 맛있고.굴비도 부드럽고, 돼지고기도 맛있네요.
항까지 태워다 주시고 넘넘 친절하십니다

22.10.14.금
랭보9
나물반찬 종류가 많고 맛있었어요
22.6.12.일
게리23
민박집을 함께 운영하고 있는 식당이다.
여사장님이 초등학교 교감선생님을 연상케 한다.까칠해 보이지만 확실하고 인정있고 깔끔하시다.
성격처럼 민박집도 이부자리도 깨끗하다.

음식은 어머니가 해 주시는 집밥처럼 맛있다

무난하고 면추가 3천원은 비싼듯
21.1.5.화
jua****
맛남
21.3.23.화
iiookk
베트남인이 운영하는 찐 베트남 맛집이예여
내부가 약간 허름하긴하지만 뭐 너무 맛있으니까
양이 많은편이 아니라 여러개 시키다보면 어느새 가격이훅하고 치고 들어와요. 가격땜시로 자주는 못먹겠지만 베트남음식 생각나면 요기로 갈꺼예요!
20.10.24.토
루꼴라페스토
잘 먹었습니다 그런데 손님안내에 신경써주셨으면 좋겠어요 그리고 살짝 비싼느낌이에요^^;
21.1.10.일
수염곰
근처 들렀다가 우연히 방문했는데 주문한 음식들 전부 맛있었어요! 양이 좀 많아서 적게 드시는 분들은 하나 다 먹기 힘들 것 같아요.
20.10.31.토
goh****
좋아요
21.3.30.화
FUTURE4
배달도돼고 맛있어요!!:) 반미가 예전처럼 빵이 부드럽지않고 약간 딱딱해진느낌인게 아쉽지만 그래도 맛있어요!!!:)
20.12.23.수
다연747
제가 먹어본 분짜중에 갑입니다!!
분짜에 짜조 나오는줄 몰라서 시켜봤는데 
주문할때 참고하세요~
근데 쌀국수는 국물이 생각보다 밍밍했어요ㅜ
그래서 별하나빼요
20.10.16.금
냐하하83
방문
21.3.3.수
이루어짐
맛잇어요
21.2.15.월
찰스세바스찬
일주일에 한번은 먹어줘야하는 맛

Hong's 평점(5점만점)
맛 4 
양 4 
위생 4 
시설,인테리어 3
대기,편의 4
주차 3
※개인적인 평가입니다 ※
21.2.16.화
제인제주
너무 맛있어요~!!
21.2.16.화
두부랑랑
쌀국수 진짜 맛있어요 👍👍 반세오도 맛있었는데 이 날은 라이스페이퍼가 너무 ... 뻣뻣해서 다 찢어지고 난리였네요 ㅠㅠ
20.11.12.목
꽃신47
리틀비엣 맛있어요 거기 볶음쌀국수 미쳤
20.11.22.일
낭만네꼬
로컬느낌 & 맛 , 비빔국수 특히 맛있네요~~
20.9.12.토
별별842
아주 맛있어요. 감사합니다
20.10.23.금
kye33
#아라동맛집 #제주베트남레스토랑 진짜 베트남 맛을 느낄수 있어요 음식 맛있어요
20.8.19.수
ear****
맛있었어요
20.

탐라가든
육류,고기요리
별점
4.42/5
제주 제주시 서사로 51
운영시간없음
064-726-1300
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더

21.11.14.일
황학동티스푼
첫 영수증 리뷰! 별점 낮게 주는 사람들 정녕 미친 건가 싶은 맛… 서울 어디서 이 가격에 이 정도 양에 이 정도 맛의 고기를 먹을 수 있을까 절대 없다 고기 다 거기서 거기고 밑반찬 정도의 차이라고 생각했었는데 탐라가든은 다르다 난 이제 처음 먹어 보는 맛이라는 감정 자체가 귀해졌는데 이건 진짜 처음 먹어 보는 맛…! 제주에 왔다면 고민하지 말고 돼지생갈비를 먹어보도록 맛이 없다면?? 별점 테러하지 말고 그냥 집에서 라면이나 끓여먹길 권해주고 싶음… 다음에 또 와야지~~!
21.7.16.금
아예24
친절해요~~
22.1.26.수
Hi77
굿
22.1.11.화
왕왕이76
너무 갈비에 진심인곳 ㅎㅎㅎ 양념도 생고기도 좋아요
21.11.6.토
살찐 올리브
친절합니다^^
22.1.26.수
지렁9918
생갈비 진짜 맛있어요!
21.12.21.화
온새미
맛있고 친절해요
21.10.23.토
SarahPark
생갈비 맛집👍🏻가격은 천원올랐네요~
생갈비는 최소 3대부터 주문가능하고 2대가나오네요^^;;;비싸서 그런지 공기밥시키면 된찌가나와요~아!멜젓이 킬포입니다👍🏻👍🏻👍🏻
21.5.27.목
가츠히로
직원분 친절하세요~ 흑돼지오겹시켰는데 잘먹었어요
21.10.7.목
달려라멍청아
좋아요
21.12.11.토
제리뽀8659
백돼지갈비 진짜 맛있어요
21.9.30.목
nkj3838
So so!!
21.12.21.화
chu****
너무 맛있어요
특히 돼지 생갈비가 좋아요
21.6.22.화
qkrwjdtn
여기는 꼭 가셔야 해요. 너무 배부른데도 너무 맛있어서 다 먹고 나왔습니다... 바싹 구워도 맛있고 부드럽게 익혀도 맛있어요ㅠㅠ 식어도 맛있는 삼겹살 드셔보셨나요..? 삼겹살에서 겉바속촉이 가능한지 몰랐는데 그걸 해냅니다. 다 구운 고기 소금 찍어서 한 입했는데 육즙인지 기름인지 팍 터지길래 감탄했어요.. 그리고 양도 진짜 많습니다. 흑돼지집가서 혼자 2인분 시켜먹고 ‘양 딱 맞네’ 하고 나왔었는데 여기는 1.5인분(3대) 시키고 진짜 버거웠어요. 여기

가게없음
이번에 찾을 키워드 : 11 / 1579 행 제주도%20슬로우카페
가게없음
이번에 찾을 키워드 : 12 / 1579 행 제주도%20노을과%20어울림%20카페
가게없음
이번에 찾을 키워드 : 13 / 1579 행 제주도%20내창생태카페
가게없음
이번에 찾을 키워드 : 14 / 1579 행 제주도%20추자북경
추자북경
중식당
별점없음
제주 제주시 추자면 신양서길 13-9 추자북경
운영시간없음
010-2260-7616
리뷰 클릭
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
['https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190731_111%2F1564535831771zCkGJ_JPEG%2FyRDXI4ojMXHyKVLbDDf75xQZ.JPG.jpg']
khj9479
두번째 방문입니다. 여전히 맛있네요. 탕수육은 꿔바로우 스타일이고 짬뽕은 담백&칼칼, 간짜장은 달지 않고 양파 식감이 너무 좋아요. 추자도 방문할 때마다 제일 먼저 생각나는 맛집이에요. 멀리 바다가 보이는 오픈된 공간에서 시원한 바람 맞으며 먹는 것도 좋고 사장님 내외분 너무 친절하세요. 올레꾼에게 최고의 힐링 스팟입니다!
22.9.25.일
104동 701호
짬뽕국물이 진짜 맛있고 탕수육도 맛있다. 가정집 같은 곳이라 찾기는 어렵지 않으나 좌석은 많지 않은 편. 재방문 의사 있음
22.11.6.일
찡빵채움
맛있어요.
아침을 안먹어 허기진데 산을 넘자마자 춘장 볶는 냄새에 ㅎㅎ
진짜 반점이 있다니,.
22.8.4.목
신오리
맛있어요!! 
22.11.24.목
twojoy
탕수육 짬뽕 짜장면 너무 맛있고, 팔씨름 하면서 재미있게 먹었습니다! ㅎ
특히, 짬뽕이 찐 맛집 인정 ㅎㅎㅎㅎ
21.9

...

글 리뷰없음
22.1.2.일
방물장수 훈도령
^^
22.5.16.월
드닝70
굿
22.1.28.금
김은지7997
잘먹었습니다~
21.12.2.목
ratmkang
깨끗하고 맛은 두말하면 잔소리~

성산일출봉 바로앞이라 주차도 편하고 깔끔하네요
21.10.20.수
안성토박이
흑돼지버거 맛있어요~~ 
빵도 토스트처럼 구운 것 같아요~ ^^
21.7.16.금
흔덕파파
매장 안이 쾌적하고 음식 비주얼이 좋았습니다.
21.11.2.화
max****
좋아요~~~
22.1.25.화
kk****
맛있어요
22.4.15.금
뜨거운호구마
인테리어가 되게 이쁘고 음식도 맛이 있었어요~
21.10.28.목
지원558
너무 맛있어요 뷰도좋고 흑돼지 육즙이 넘치네요~
21.10.26.화
류진아
매장이 예쁘고 음식이 정성스럽네요.맛있어요^^
21.10.25.월
누 똑 시
직접 만들어주신 수제버거가 푸짐하고 맛나네요.특히 패티가 두툼해서 넘 좋았어요.테이크아웃해갔는데 포장도 깔끔하고요~~
21.8.12.목
me0****
제주도 성산일출봉
22.1.15.토
LuciA94
성산일출봉 맛집이네요
21.11.21.일
Happyfactory
글 리뷰없음
21.11.24.수
이예원8912
맛있어요!!
21.12.26.일
hay****
성산일출봉 바로앞이라 들르기 좋아용
21.9.20.월
phj****
맛있어요
21.12.25.토
als****
깨끗하고 뷰맛집입니다. 
맛있어요
21.10.21.목
hyu****
상호도 예쁘고 성산일출봉 주차장에서 들어가는 길도 넘흐 예뻤어요. 
치킨,떡갈비 주문하고 맛있는 수제버거를 생각하고 기다렸는데...ㅜ 버거 패티가 기름에 완젼 발린 느낌... 그리고 많이 짰습니다. 수제버거라 넘흐 기대했었나봐요.
21.8.11.수
usayss
글 리뷰없음
21.12.1.수
함쓰97
성산일출봉 보고 내려왔다가 냄새에 홀려 들어왔어요
21.10.20.수
LOADING99
성산일출봉 올라갔다 내려와서 먹어요
21.10.19.화
부루스뤼
햄버거 맛없음
커피 맛없음
21.10.19.화


리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
['https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20170515_193%2F1494823455128CXX3k_JPEG%2F186465537431936_0.jpeg']
HonesT
지인 소개로 한번 먹어보고싶다 그래서 가봤는데 
일단 시간대가 저녁시간은 아니였는데 금방 풀 차더라구요 예약하고 가는것도 추천이여 ㅎㅎ 바닥에 앉는곳도 그리고 그냥 의자에 앉는 곳도 있어서 좋아요 그리고 솔직히 진짜 기대도 안하고 갔는데 
오 문어로 여러가지 음식을 할수있네? 라고 느끼게 해준 곳이에요 무침은 뭐 흔하다고 하지만 
옆에 꼬치가 진짜 제일 맛있었어요 버터에다가 
야채들이랑 다같이 해가지고 진짜 다 맛있었는데 
저게 특히 제일 맛있더라구요 그리고 기본찬도 너무 잘 나와서 감탄했네요 기본찬도 하나도 빠짐없이 다 먹었어요 다 맛있어서 안먹을수가 없어요 ㅋㅋㅋㅋ그리고 사장님분들도 친절하시고 착하셔요 ㅎㅎ 
가격은 비싼것도 싼것도 아닌 진짜 딱 좋은거같아요 
다음에 또 가야겠어요 
1.9.월
카라63
오늘은 우럭조림 먹고싶어서 왔어요.
문어요리 전문점 이지만 우럭조림도 맛있습니다.우럭조림은 먹는데 집중해서 사진을 못찍

가게없음
이번에 찾을 키워드 : 240 / 1579 행 제주도%20유복현식공간
가게없음
이번에 찾을 키워드 : 241 / 1579 행 제주도%20채식카페%20제주작은부엌
가게없음
이번에 찾을 키워드 : 242 / 1579 행 제주도%20크라운돼지
가게없음
이번에 찾을 키워드 : 243 / 1579 행 제주도%20페를로
페를로
이탈리아음식
별점
4.53/5
제주 서귀포시 안덕면 덕수회관로74번길 33
운영시간없음
0507-1396-9501
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클

더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭

더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭
더보기 클릭

In [ ]:
name_list